In [1]:
!pip install pinecone-client cohere transformers sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.4/222.4 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00


In [2]:
import pinecone
from sentence_transformers import SentenceTransformer
import cohere



In [3]:
import os
from pinecone import Pinecone, ServerlessSpec


pc = Pinecone(api_key="7ea2842e-0423-4c36-a409-79889f26a01d")
index_name = "qa-bot-index"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=768, 
        metric='cosine',  
        spec=ServerlessSpec(
            cloud='aws', 
            region='us-east-1' 
        )
    )

index = pc.Index(index_name)

In [4]:
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

documents = ["Document 1 content...", "Document 2 content..."]

doc_embeddings = model.encode(documents)


ids = [f"doc_{i}" for i in range(len(documents))]
index.upsert(vectors=zip(ids, doc_embeddings))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

{'upserted_count': 2}

In [5]:
query = "What is the content of Document 1?"
query_embedding = model.encode([query])[0]


In [7]:

result = index.query(
    vector=query_embedding.tolist(),  
    top_k=5
)

print("Query Result:", result)

retrieved_docs = []
for match in result['matches']:
    doc_id = match['id']  

    if doc_id.startswith("doc_"):
        try:
            index = int(doc_id.split('_')[1]) 
            if 0 <= index < len(documents):  
                retrieved_docs.append(documents[index])  
            else:
                retrieved_docs.append("Index out of range")  
        except ValueError:
           
            retrieved_docs.append(f"Unknown ID format: {doc_id}")
    else:
        
        retrieved_docs.append(f"Unexpected ID format: {doc_id}")

print("Retrieved Documents:", retrieved_docs)

Query Result: {'matches': [{'id': 'doc_0', 'score': 0.845988393, 'values': []},
             {'id': 'doc_1', 'score': 0.810611963, 'values': []},
             {'id': 'uploaded_doc', 'score': 0.135064289, 'values': []}],
 'namespace': '',
 'usage': {'read_units': 5}}
Retrieved Documents: ['Document 1 content...', 'Document 2 content...', 'Unexpected ID format: uploaded_doc']


In [8]:
cohere_client = cohere.Client("Ga9HyHd21Nd5XoiXxdSiPzsmtGZJhULrfuS8gfct")


In [15]:
import pinecone
from sentence_transformers import SentenceTransformer

pinecone.init(api_key="7ea2842e-0423-4c36-a409-79889f26a01d", environment="us-east-1")
index_name = "qa-bot-index"


index = pinecone.Index(index_name)

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

documents = ["Document 1 content...", "Document 2 content..."]


doc_embeddings = model.encode(documents)


ids = [f"doc_{i}" for i in range(len(documents))]
index.upsert(vectors=[(ids[i], doc_embeddings[i], {'text': documents[i]}) for i in range(len(documents))])

user_query = "What is html?"
query_embedding = model.encode(user_query)

result = index.query(vector=query_embedding.tolist(), top_k=3)

print("Query result:", result)

retrieved_documents = []
for match in result['matches']:
    doc_id = match['id']
    print("Document ID:", doc_id) 
    try:
        index = int(doc_id.split('_')[1])
        retrieved_document = documents[index]
        retrieved_documents.append(retrieved_document)
    except (IndexError, ValueError) as e:
        print(f"Error processing document ID '{doc_id}': {e}")

print("Retrieved Documents:", retrieved_documents)


AttributeError: init is no longer a top-level attribute of the pinecone package.

Please create an instance of the Pinecone class instead.

Example:

    import os
    from pinecone import Pinecone, ServerlessSpec

    pc = Pinecone(
        api_key=os.environ.get("PINECONE_API_KEY")
    )

    # Now do stuff
    if 'my_index' not in pc.list_indexes().names():
        pc.create_index(
            name='my_index', 
            dimension=1536, 
            metric='euclidean',
            spec=ServerlessSpec(
                cloud='aws',
                region='us-west-2'
            )
        )



In [16]:
import pinecone
from sentence_transformers import SentenceTransformer
import cohere

pc = pinecone.Pinecone(api_key="7ea2842e-0423-4c36-a409-79889f26a01d")

index_name = "qa-bot-index"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=768,  
        metric='cosine',  
        spec=pinecone.ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )

index = pc.Index(index_name)

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

documents = ["Document 1 content...", "Document 2 content..."]

doc_embeddings = model.encode(documents)

ids = [f"doc_{i}" for i in range(len(documents))]
index.upsert(vectors=[(ids[i], doc_embeddings[i], {'text': documents[i]}) for i in range(len(documents))])


user_query = "What is html?"
query_embedding = model.encode(user_query)

result = index.query(vector=query_embedding.tolist(), top_k=3)

print("Query result:", result)
retrieved_documents = []
for match in result['matches']:
    doc_id = match['id']
    print("Document ID:", doc_id)  
    try:
        index = int(doc_id.split('_')[1])
        retrieved_document = documents[index]
        retrieved_documents.append(retrieved_document)
    except (IndexError, ValueError) as e:
        print(f"Error processing document ID '{doc_id}': {e}")

print("Retrieved Documents:", retrieved_documents)


cohere_client = cohere.Client("Ga9HyHd21Nd5XoiXxdSiPzsmtGZJhULrfuS8gfct")

prompt = f"""
The following are excerpts from the relevant documents:

{retrieved_documents[0]}
"""

if len(retrieved_documents) > 1:
    prompt += f"""{retrieved_documents[1]}
    """
if len(retrieved_documents) > 2:
    prompt += f"""{retrieved_documents[2]}
    """

prompt += f"""
Based on the provided excerpts, answer the following question:
{user_query}
"""

response = cohere_client.generate(
    model='command-xlarge-nightly',
    prompt=prompt,
    max_tokens=150
)

print("Answer:", response.generations[0].text)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Query result: {'matches': [{'id': 'doc_0', 'score': 0.28855139, 'values': []},
             {'id': 'doc_1', 'score': 0.252192944, 'values': []},
             {'id': 'uploaded_doc', 'score': 0.187980503, 'values': []}],
 'namespace': '',
 'usage': {'read_units': 5}}
Document ID: doc_0
Document ID: doc_1
Document ID: uploaded_doc
Error processing document ID 'uploaded_doc': invalid literal for int() with base 10: 'doc'
Retrieved Documents: ['Document 1 content...', 'Document 2 content...']
Answer: HTML (Hypertext Markup Language) is the code that is used to structure and display web content in a web browser. It defines the meaning and structure of web content, and is used to create paragraphs, links, headings, lists, embedded audio and video, and other elements. HTML uses "markup" to annotate text, images, and other content in order to display it in different ways.


In [17]:
response = cohere_client.generate(
    model='command-xlarge-nightly',  
    prompt=prompt,
    max_tokens=150
)
print(response.generations[0].text)


HTML (Hypertext Markup Language) is the code that is used to structure and display web content in a web browser. It is the standard markup language for creating web pages. HTML consists of a series of elements, which you use to enclose, wrap, or mark up different parts of the content to make it appear or act a certain way. The elements are represented by tags. Some tags require an opening and closing tag (<html></html>), while others are standalone (such as the line break tag <br>).


In [20]:
import pinecone
from sentence_transformers import SentenceTransformer


pc = pinecone.Pinecone(api_key="7ea2842e-0423-4c36-a409-79889f26a01d")


index_name = "qa-bot-index"


if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=768,  
        metric='cosine',  
        spec=pinecone.ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )


index = pc.Index(index_name)


model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

documents = ["Document 1 content...", "Document 2 content..."]


embeddings = model.encode(documents)

ids = [f"doc-{i}" for i in range(len(documents))]


index.upsert(vectors=zip(ids, embeddings))


{'upserted_count': 2}